# 接口：从协议到抽象基类

> ”抽象类表示接口“ --- C++之父

联想到 C++ 中基类的虚函数，只定义方法，但不实现方法。

## 1、Python 文化中的接口和协议

类中的常规变量和方法都是 public 属性。

带有双下划线的变量和方法才是 private。
- \_\_x
- \_\_func

接口：对象公开方法的子集，让对象在系统中扮演的特定角色。一个类可能实现多个接口，从而让实例扮演多个角色。

## 2、Python喜欢序列

Python 会特殊看待像是序列的对象，会尽量的调用已有方法满足不同的方法。

## 3、使用猴子补丁在运行时实现协议

猴子补丁的定义：在运行时动态修改类和模块，而不改动源码。

In [12]:
class FrenchDeck:
    def __init__(self):
        self._cards = [1, 2, 3]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

import random

deck = FrenchDeck()

random.shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

**对象默认的是不可变序列，可变序列必须提供 \_\_setitem\_\_ 方法。**

为了解决上述问题，可以实现 FrenchDeck 的 \_\_setitem\_\_ 的方法，或者由于 Python 是动态语言，可以交互式的定义 \_\_setitem\_\_ 方法。

In [24]:
def set_card(deck, position, card):
    deck._cards[position] = card
FrenchDeck.__setitem__ = set_card
random.shuffle(deck)
deck[:]

[3, 2, 1]

注：特殊 \_\_setitem\_\_ 方法在 Pytho 参考手册中的参数是 (self, key, value)，而这里使用 (deck, position, card)。

**说明了，每个 Python 方法到底都是普通函数，把第一个参数命名为 self 只是一种约定。**

## 4、Alex Martelli 的水禽

下面是讲解抽象基类。

In [16]:
from abc import ABC, ABCMeta
class A():
    def __init__(self):
        pass
a = A()

In [11]:
isinstance(a, A)

True

## 5、定义抽象基类的子类

一个必须遵守的规则：利用现有的抽象基类。

声明抽象基类的子类的方法，就是类声明的时候，注明父类为抽象基类。

## 6、标准库中的抽象基类

讨论抽象基类通常不用考虑多重继承。

### abc 包中的 ABC

collection.abc 模块定义了16个抽象基类。

MixIn混入方法：各个基类提供了抽象方法和具体方法。

## 7、定义并使用一个抽象基类

场景：创建一个”随机挑选的无重复“的抽象基类(组件)。

抽象基类命名为 Tombola，一共有四个方法，以下两个是抽象方法：
- load(...): 把元素放入容器。
- pick(): 从容器中随机拿出一个元素，返回选中的元素。

另外两个是具体方法：
- loaded(): 如果容器总至少有一个元素，返回True。
- inspect: 返回一个有序元组，有容器内的现有元素构成。

Tombola抽象基类和三个具体实现。

![抽象基类-Tombola.jpg](./image/抽象基类-Tombola.jpg)

下面是 Tombola 抽象基类的定义。

In [19]:
import abc
class Tombola(abc.ABC): # 自己定义的抽象基类要继承abc.ABC。
    @abc.abstractmethod
    def load(self, iterable):
        """add element from iterable."""
        
    @abc.abstractmethod # 抽象方法使用@abstractmethod装饰器标记，而且定义体中通常只有文档字符串。
    def pick(self):
        """random delete element, and return element."""
    
    def loaded(self): # 抽象基类可以包含具体方法。
        """return ture if element number > 1, else false."""
        return bool(self.inspect())
    
    def inspect(self): # 抽象基类中的具体方法只能依赖抽象基类定义的接口（即只能使用抽象基类中的其他具体方法、抽象方法或特性）。
        """return ordered tuple"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

**其实，抽象方法可以有实现代码。即便实现了，子类也必须覆盖抽象方法，但是在子类中可以使用super（　）函数调用抽象方法，为它添加功能，而不是从头开始实现。**

下面展示不符合 Tombola 要求的子类无法蒙混过关。

In [9]:
class Fake(Tombola):
    def pick(self):
        return 13
Fake # No Error
f = Fake() # instance will report Error.

TypeError: Can't instantiate abstract class Fake with abstract method load

### 抽象基类句法详解

声明抽象基类最简单的方式是继承abc.ABC或其他抽象基类。

abc.ABC 是 Python3.4 新增的类。Python3.4之前，声明抽象基类的办法是在 class 语句中使用 metaclass=abc.ABCMeta (不是abc.ABC)。

In [14]:
class Tombola(metaclass=abc.ABCMeta):
    # ...

metaclass 特性是 Python3 引入的，Python2中必须使用 __metaclass__ 类属性。

In [17]:
class Tombola(object):
    __metaclass__ = abc.ABC

**函数装饰器堆叠。**

比如声明一个类的方法为 @classmethod、@abc.abstractmethod，这时引入一个新的问题，函数装饰器堆叠顺序。

一条重要的规则是，与其他方法描述符一起使用时，abstractmethod（　）应该放在最里层，也就是说，在@abstractmethod和def语句之间不能有其他装饰器。

### 定义 Tombola 抽象基类的子类

子类实现抽象基类的所有抽象方法。

**一个深拷贝的技巧**：balls = list(\_balls)，这样会创建 \_balls 的副本，这个技巧在 class 的 \_\_init\_\_ 函数被用到，balls 是类变量，\_balls 是方法的参数。

### Tombola 的虚拟子类

class_name.register 装饰器在不继承的情况下，把一个类注册为抽象基类的**虚拟子类**(与普通的子类不同)。

注册虚拟子类的方式是在抽象基类上调用register方法。这么做之后，注册的类会变成抽象基类的虚拟子类，而且issubclass和isinstance等函数都能识别，但是注册的类不会从抽象基类中继承任何方法或属性。

虚拟子类不会继承注册的抽象基类，而且任何时候都不会检查它是否符合抽象基类的接口，即便在实例化时也不会检查。为了避免运行时错误，虚拟子类要实现所需的全部方法。

虚拟子类只是宣称可以向虚拟基类那样使用。

**一个特殊的类属性**: __mro__，记录了类的继承关系。

用法：TombolaList.\_\_mro\_\_。

注意，TombolaList.\_\_mro\_\_ 不包含class_name.register 装饰器注册的抽象基类。

### Tombola 子类的测试方法

\_\_subclass\_\_(): 返回类的直接子类列表，不含虚拟子类。

In [28]:
Tombola.__subclasses__()

[__main__.Fake]

\_abc\_registry？